1. Delete the columns Time, Sub_metering_2 and Sub_metering_3

In [5]:
import pandas as pd

data = pd.read_csv(
    "../data/household_power_consumption.txt", 
    sep=";", 
    low_memory =False
)

data_drop = data.drop(['Time', 'Sub_metering_2', 'Sub_metering_3'], axis=1)

print(data_drop)

             Date Global_active_power Global_reactive_power  Voltage  \
0      16/12/2006               4.216                 0.418  234.840   
1      16/12/2006               5.360                 0.436  233.630   
2      16/12/2006               5.374                 0.498  233.290   
3      16/12/2006               5.388                 0.502  233.740   
4      16/12/2006               3.666                 0.528  235.680   
...           ...                 ...                   ...      ...   
65639   31/1/2007               0.698                 0.134  240.810   
65640   31/1/2007               0.706                 0.134  240.600   
65641   31/1/2007               0.698                 0.122  241.600   
65642   31/1/2007               0.638                 0.000  241.460   
65643   31/1/2007                 NaN                   NaN      NaN   

      Global_intensity Sub_metering_1  
0               18.400          0.000  
1               23.000          0.000  
2              

2. Set Date as index

In [41]:
df0 = pd.DataFrame(data_drop)
df_indexed = df0.set_index('Date')

print(df_indexed)

           Global_active_power Global_reactive_power  Voltage  \
Date                                                            
16/12/2006               4.216                 0.418  234.840   
16/12/2006               5.360                 0.436  233.630   
16/12/2006               5.374                 0.498  233.290   
16/12/2006               5.388                 0.502  233.740   
16/12/2006               3.666                 0.528  235.680   
...                        ...                   ...      ...   
31/1/2007                0.698                 0.134  240.810   
31/1/2007                0.706                 0.134  240.600   
31/1/2007                0.698                 0.122  241.600   
31/1/2007                0.638                 0.000  241.460   
31/1/2007                  NaN                   NaN      NaN   

           Global_intensity Sub_metering_1  
Date                                        
16/12/2006           18.400          0.000  
16/12/2006         

Create a function that takes as input the DataFrame with the data set and returns a DataFrame with updated types:

```
    def update_types(df):
        #TODO
        return df

In [91]:
import pandas as pd

def update_types(df):
    """
    Update DataFrame column types to more appropriate data types.
    
    This function attempts to:
    - Convert numeric strings to numeric types (if majority are numbers)
    - Convert date strings to datetime
    - Convert low-cardinality string columns to categorical
    - Optimize integer types to smaller sizes where possible
    - Update index column types as well
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame with columns to optimize
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with optimized column types
    """
    df = df.copy()
    
    # Helper function to check if value can be numeric
    def is_numeric_value(val):
        if pd.isna(val):
            return True  # NaN is acceptable for numeric
        try:
            float(val)
            return True
        except (ValueError, TypeError):
            return False
    
    # Update index column
    if df.index.dtype == 'object':
        # Check if majority of index values are numeric
        numeric_count = sum(is_numeric_value(val) for val in df.index)
        total_count = len(df.index)
        
        if numeric_count / total_count > 0.5:  # More than 50% are numeric
            try:
                df.index = pd.to_numeric(df.index, errors='coerce')
            except:
                pass

        if df.index.dtype == 'object':
            df.index = pd.to_datetime(df.index, errors='coerce', dayfirst=True)
        
    # Update data columns
    for col in df.columns:
        col_type = df[col].dtype
        
        # Convert object columns
        if col_type == 'object':
            # Try to convert to numeric
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                pass

        if df[col].dtype == 'object':
            numeric_count = sum(is_numeric_value(val) for val in df[col])
            total_count = len(df[col])
            
            if numeric_count / total_count > 0.5:  # More than 50% are numeric
                try:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                except:
                    pass

            # If still object, try datetime
            if df[col].dtype == 'object':
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
                except:
                    pass
            
            # If still object and low cardinality, convert to category
            if df[col].dtype == 'object':
                num_unique = df[col].nunique()
                num_total = len(df[col])
                if num_unique / num_total < 0.5:  # Less than 50% unique values
                    df[col] = df[col].astype('category')
        
        # Optimize integer types
        elif col_type in ['int64', 'int32']:
            col_min = df[col].min()
            col_max = df[col].max()
            
            if col_min >= 0:
                if col_max < 255:
                    df[col] = df[col].astype('uint8')
                elif col_max < 65535:
                    df[col] = df[col].astype('uint16')
                elif col_max < 4294967295:
                    df[col] = df[col].astype('uint32')
            else:
                if col_min > -128 and col_max < 127:
                    df[col] = df[col].astype('int8')
                elif col_min > -32768 and col_max < 32767:
                    df[col] = df[col].astype('int16')
                elif col_min > -2147483648 and col_max < 2147483647:
                    df[col] = df[col].astype('int32')
        
        # Optimize float types
        elif col_type == 'float64':
            df[col] = df[col].astype('float32')
    
    return df

Use describe to have an overview on the data set

In [92]:
df_updated = update_types(df_indexed)

df_updated.describe(percentiles=None, include=None, exclude=None)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1
count,65637.000000,65637.000000,65637.000000,65637.000000,65637.000000
mean,1.655605,0.132055,241.117835,7.004540,1.278136
std,1.345990,0.118482,3.624191,5.673438,6.655240
min,0.194000,0.000000,224.680000,0.800000,0.000000
25%,0.394000,0.000000,238.670000,1.800000,0.000000
50%,1.424000,0.122000,241.420000,6.000000,0.000000
75%,2.428000,0.198000,243.730000,10.000000,0.000000
max,9.272000,0.874000,251.700000,40.400000,77.000000


5. Delete the rows with missing values

In [93]:
df_clean = df_updated.dropna()

print(df_clean)

            Global_active_power  Global_reactive_power  Voltage  \
Date                                                              
2006-12-16                4.216                  0.418   234.84   
2006-12-16                5.360                  0.436   233.63   
2006-12-16                5.374                  0.498   233.29   
2006-12-16                5.388                  0.502   233.74   
2006-12-16                3.666                  0.528   235.68   
...                         ...                    ...      ...   
2007-01-31                0.700                  0.136   240.84   
2007-01-31                0.698                  0.134   240.81   
2007-01-31                0.706                  0.134   240.60   
2007-01-31                0.698                  0.122   241.60   
2007-01-31                0.638                  0.000   241.46   

            Global_intensity  Sub_metering_1  
Date                                          
2006-12-16              18.4      

6. Modify Sub_metering_1 by adding 1 to it and multiplying the total by 0.06. If x is a row the output is: (x+1)*0.06

In [94]:
# type(df_clean['Sub_metering_1'].iloc[0])
# df_clean.loc[:, 'Sub_metering_1'] = (df_clean['Sub_metering_1'] + 1) * 0.06

df_updated = df_clean.copy()
df_updated['Sub_metering_1'] = (df_updated['Sub_metering_1'] + 1) * 0.06
print(df_updated)

            Global_active_power  Global_reactive_power  Voltage  \
Date                                                              
2006-12-16                4.216                  0.418   234.84   
2006-12-16                5.360                  0.436   233.63   
2006-12-16                5.374                  0.498   233.29   
2006-12-16                5.388                  0.502   233.74   
2006-12-16                3.666                  0.528   235.68   
...                         ...                    ...      ...   
2007-01-31                0.700                  0.136   240.84   
2007-01-31                0.698                  0.134   240.81   
2007-01-31                0.706                  0.134   240.60   
2007-01-31                0.698                  0.122   241.60   
2007-01-31                0.638                  0.000   241.46   

            Global_intensity  Sub_metering_1  
Date                                          
2006-12-16              18.4      

7. Select all the rows for which the Date is greater or equal than 2008-12-27 and Voltage is greater or equal than 242

In [126]:
type(df_updated.index.to_series().iloc[0])

start_date='2008-12-27'
min_voltage=242

start_date = pd.to_datetime(start_date)

filtered_df = df_updated[
        (df_updated.index >= start_date) & 
        (df_updated['Voltage'] >= min_voltage)
    ]

df_filtered.dropna()

print(df_filtered)

pandas._libs.tslibs.timestamps.Timestamp